**Data Normalization & Dupe Remover for Singapore Data**

In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------- ------------------------------ 2.9/12.8 MB 15.1 MB/s eta 0:00:01
     ------------- -------------------------- 4.5/12.8 MB 11.5 MB/s eta 0:00:01
     ------------------- -------------------- 6.3/12.8 MB 10.2 MB/s eta 0:00:01
     ----------------------- ---------------- 7.6/12.8 MB 9.4 MB/s eta 0:00:01
     ---------------------------- ----------- 9.2/12.8 MB 8.9 MB/s eta 0:00:01
     -------------------------------- ------- 10.5/12.8 MB 8.8 MB/s eta 0:00:01
     -------------------------------------- - 12.3/12.8 MB 8.6 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 8.5 MB/s  0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import pandas as pd
import glob
import re

In [3]:
import spacy

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

In [4]:
# Combine All of the data into a unified whole

country = "Thailand"

all_files = glob.glob(f"../data/{country}_Job-Listing/*.csv")

dfs = []
for file in all_files:
    df = pd.read_csv(file)

    # extract keyword from filename
    keyword = file.split("__")[1].replace("_final", "").replace(".csv", "").replace("-", " ")

    df["search_keyword"] = keyword.lower()
    df["source_file"] = file
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

In [5]:
# combined_df.tail(10)

In [6]:
combined_df["search_keyword"].value_counts()

search_keyword
data analyst                          1706
data scientist                         885
data engineer                          627
sql developer                          624
machine learning engineer              347
ai strategist                          266
analytics consultant                   199
ai researcher                          182
deep learning engineer                 181
data science consultant                181
power bi analyst                       170
tableau developer                      164
data product manager                   158
business intelligence analyst          131
ai engineer                            126
data architect                         114
nlp engineer                           101
artificial intelligence specialist      90
data driven decision maker              78
generative ai developer                 72
ai product manager                      42
computer vision specialist              42
data strategy consultant               

In [ ]:
abbr_dict = {
    "ai": {"artificial", "intelligence"},
    "ml": {"machine",  "learn"},
    "nlp": {"natural",  "language", "process"},
    "qa": {"quality",  "assurance"},
    "swe": {"software", "engineer"},
    "llm": {"large", "language", "model"},
    "cybersecurity": {"cyber", "security"},
    
    
}

job_title_mapping = {
    "ai engineer": {
        "alternatives": [
            {
                "required": {"artificial", "intelligence"},
                "any_of": {"engineer", "developer", "architect"},
            }
        ]  
    },
    "ai product manager": {"artificial", "intelligence", "product", "manager"},
    "ai researcher": {
        "alternatives": [
            {
                "required": {"artificial", "intelligence"},
                "any_of": {"science", "research"},
            },
            {
                "required": {"machine", "learning"},
                "any_of": {"science", "research"},
            },
            {
                "required": {"large", "model"},
                "any_of": {"science", "research"},
            },
            # large, language, model
            {
                "required": {"nlp"},
                "any_of": {"science", "research"},
            },
            {
                "required": {"genai"},
                "any_of": {"science", "research"},
            }
        ]
        
    },
    "ai strategist": {
        "alternatives": [
            {
                "required": {"artificial", "intelligence"},
                "any_of": {"strategy", "governance", "planning"}
            },
            {
                "required": {"artificial", "intelligence", "risk", "management"},
            }
        ]
    },
    "artificial intelligence specialist": {"artificial", "intelligence", "specialist"},
    "analytics consultant": {"analytics", "consultant"},
    "business intelligence analyst": {
        "required": {"business", "intelligence"},
        "any_of": {"analyst", "specialist"},
    },
    "data analyst": {
        "required": {"data"},
        "any_of": {"analytics", "analyst", "specialist"},
        # intelligence
    },
    "data architect": {"data", "architect"},
    "data driven decision maker": {"data", "driven", "decision", "maker"},
    "data engineer": {
        "alternatives": [
            {
                "required": {"data"},
                "any_of": {"engineer", "modeler"}
            },
            {
                "required": {"data", "platform"},
                "any_of": {"engineer", "modeler", "developer"}
            },
            {
                "required": {"data", "warehouse"},
                "any_of": {"engineer", "modeler", "developer"}
            },
            {
                "required": {"ETL"},
                "any_of": {"engineer", "modeler", "developer"}
            }

        ]
        
    },
    "data product manager": {
        "alternatives": [
            {
                "required": {"data", "product"},
                "any_of": {"manager", "director"},
            },
            {
                "required": {"data", "project"},
                "any_of": {"manager", "director"},
            }
        ]
    },
    
    # "data research": {"data", "research"},
    "data science consultant": {"data", "science", "consultant"},
    "data scientist": {"data", "science"},
    "data strategy consultant": {"data", "strategy"},
    "data visualization": {
        "alternatives": [
            {
                "required": {"data", "visualization"},
                "any_of": {"visualization", "analyst", "tool", "developer", "specialist"}
            },
            {
                "required": {"power", "bi"},
                "any_of": {"visualization", "analyst", "tool", "developer", "specialist"}
            },
            {
                "required": {"tableau"},
                "any_of": {"visualization", "analyst", "tool", "developer", "specialist"}
            }
        ]
    },
    "database engineer": {
        "required": {"database"},
        "any_of": {"engineer", "administration", "analyst"},
    },
    "deep learning engineer": {"deep", "learn"},
    "computer vision specialist": {
        "required": {"vision"},
        "any_of": {"specialist", "computer", "engineer" },
    },
    "LLM / GenAI": {
        "alternatives": [
            {
                "required": {"artificial", "intelligence"},
                "any_of": {"generative", "gen"},
            },
            {
                "required": {"genai"},
                "any_of": {"developer", "engineer", "architect"},
            },
            {
                "required": {"large", "language", "model"},
                "any_of": {"developer", "engineer", "architect"},
            }
        ]
    },
    "machine learning engineer": {
        "alternatives": [
            {
                "required": {"machine", "learn"},
                "any_of": {"engineer"},
            },
            {
                "required": {"mlops"},
                "any_of": {"engineer"},
            },
            {
                "required": {"mllm"},
                "any_of": {"engineer"}
            },
        ]
    },  
    "nlp engineer": {"natural", "language", "process", "engineer"},

    "sql developer": {
        "required": {"developer"},
        "any_of": {"postgressql", "mysql", "sql", "oracle", "plsql"},
    },
    "sql engineer": {
        "required": {"engineer"},
        "any_of": {"postgressql", "mysql", "sql", "oracle", "plsql"},
    },
}


other_job_title_mapping = {
    "Other specialist: data governance": {
        # Additional Title Classification
        "required": {"data"},
        "any_of": {"governance", "steward", "management"},
    },
    "Other specialist: data product designer": {
        # Additional Title Classification
        "data", "product", "designer"
    },
    "Other specialist: devOps": {
        # Additional Title Classification
        "required": {"devops"},
        "any_of": {"developer", "engineer", "business"}
    },
    "Other specialist: cloud specialist": {
        # Additional Title Classification
        "alternatives": [
            {
                "required": {"cloud"},
                "any_of": {"architect", "consultant" "developer", "engineer", "operation", "specialist"}
            },
            {
                "required": {"aws"},
                "any_of": {"architect", "consultant" "developer", "engineer", "operation", "specialist"}
            },
            {
                "required": {"azure"},
                "any_of": {"architect", "consultant" "developer", "engineer", "operation", "specialist"}
            },
        ]
    },
    "Other specialist: automation": {
        "alternatives": [
            {
                "required": {"rpa"},
            },
            {
                "required": {"vba"},
            }
        ]
    },
    "Other specialist: cybersecurity": {
        # Additional Title Classification
        "alternatives": [
            {
                "required": {"penetration", "test"},
            },
            {
                "required": {"cyber", "security"},
                "any_of": {"architect", "consultant" "developer", "engineer", "operation", "specialist"}
            },
            {
                "required": {"security"},
                "any_of": {"architect", "consultant" "developer", "engineer", "operation", "specialist"},
            }
        ]
    },

}



job_variant_mapping = {
    "administrator": "administration",
    "analysts": "analyst", "analysis": "analyst", "analyzing": "analyst",
    "architecture": "architect", "architecting": "architect",
    "consultants": "consultant", "consulting": "consultant",
    "developers": "developer", "developing": "developer", "development": "developer",
    "engineers": "engineer", "engineering": "engineer",
    "learning": "learn", "learns": "learn", "learned": "learn",
    "managers": "manager", "managing": "manager", "mangement": "manager",
    "operations": "operation", "operating": "operation",
    "researchers": "research", "researcher": "research", "researching": "research",
    "scientist": "science", "scientists": "science", "scientifying": "science",
    "specialists": "specialist", "specializing": "specialist",
    "strategists": "strategy", "strategizing": "strategy", "strategies": "strategy",
    "testers": "test", "testing": "test",
}

# Words to ignore for core roles
noise_words = {"based", "contract", "english", "fluency", "grad",
                "intern", "internship", "junior", "lead", "level",
                "mid", "new", "remote", "required", "senior", "welcome",
                "enterprise", "urgent"
            }


In [ ]:
def normalize_job_title(title):

    # Tokenize the title 
    doc = nlp(title.lower())

    # List to store normalized tokens
    tokens_set = set()

    for token in doc:

        if token.is_punct or token.is_space or token.is_stop:
            continue
        
        # Get both original and lemma forms
        original_word = re.sub(r"[-_.,(){}\[\]/]", " ", token.text.lower())
        lemma_word = re.sub(r"[-_.,(){}\[\]/]", " ", token.lemma_.lower())


        # Check if the word is an abbreviation or a variant
        for word in {original_word, lemma_word}:
            for w in word.split():
                # Ignore noise words
                if not w or w in noise_words:
                    continue

                # Expand abbreviations
                if w in abbr_dict:
                    tokens_set.update(abbr_dict[w])

                # Check if the word is a variant
                elif w in job_variant_mapping:
                    tokens_set.add(job_variant_mapping[w])
                else:
                    tokens_set.add(w)

    print(title, "->", tokens_set)
    return tokens_set

def match_job_title(tokens_set, mapping):
    # Alternative mapping
    if isinstance(mapping, dict) and "alternatives" in mapping:
        for option in mapping["alternatives"]:
            required = option.get("required", set())
            any_of = option.get("any_of", set())
            if required.issubset(tokens_set) and (not any_of or tokens_set & any_of):
                return True
        return False

    
    # Flexible mapping
    if isinstance(mapping, dict):
        required = mapping.get("required", set())
        any_of = mapping.get("any_of", set())

        # All required must be present, at least one of any_of (if any_of not empty)
        if required.issubset(tokens_set) and (not any_of or tokens_set & any_of):
            return True

    # Strict mapping
    elif isinstance(mapping, set):
        if mapping.issubset(tokens_set):
            return True

    return False



def normalize_title(row, title_cache):
    search_kw = row["search_keyword"].lower()
    raw_words_set = title_cache[row["title"]]

    # --- Step 1: Search keyword priority ---
    if search_kw in job_title_mapping:
        mapping = job_title_mapping[search_kw]

        if match_job_title(raw_words_set, mapping):
            return search_kw

    # --- Step 2: Fallback: check job title text ---
    for normalized, mapping in job_title_mapping.items():
        if match_job_title(raw_words_set, mapping):
            return normalized

    # --- Step 3: Nothing matched ---
    return "Unclassified"


# --- FAST NORMALIZATION: pre-process unique titles ---
unique_titles = combined_df["title"].unique()
title_cache = {title: normalize_job_title(title) for title in unique_titles}

combined_df["normalized_title"] = combined_df.apply(lambda row: normalize_title(row, title_cache), axis=1)
# add a parameter for job title mapping

# then do the combine_df again, but this time, only for those with unclassified titles, and do the otehr mapping this time


AI Engineer -> {'engineer', 'artificial', 'intelligence'}
AI Engineer (Contract 1 Year) - EGG Digital -> {'engineer', 'contract', 'egg', 'artificial', 'year', 'digital', '1', 'intelligence'}
AI Engineer (Pathum Thani) -> {'engineer', 'pathum', 'artificial', 'thani', 'intelligence'}
Responsible AI Engineer -> {'engineer', 'artificial', 'intelligence', 'responsible'}
Senior AI Engineer (EGG Digital) -> {'engineer', 'egg', 'artificial', 'digital', 'intelligence'}
Machine Learning Engineer (Project Algorithmic Trading) -> {'engineer', 'machine', 'project', 'algorithmic', 'learn', 'trading'}
AI Engineer (Innovation Department) -> {'engineer', 'department', 'artificial', 'innovation', 'intelligence'}
AI Solutions Analyst -> {'solution', 'artificial', 'analyst', 'solutions', 'intelligence'}
Data Engineer (AI Engineer) -> {'data', 'engineer', 'artificial', 'intelligence'}
Data Scientist (AI Engineer) -> {'engineer', 'science', 'artificial', 'data', 'intelligence'}
AI/ML Engineer -> {'engineer'

In [9]:
combined_df["normalized_title"].value_counts()

normalized_title
Unclassified                          4298
data analyst                           637
ai engineer                            401
data engineer                          381
data scientist                         309
database engineer                       84
machine learning engineer               69
artificial intelligence specialist      67
business intelligence analyst           58
ai researcher                           46
data architect                          45
data product manager                    29
sql developer                           29
data visualization                      25
analytics consultant                    12
LLM / GenAI                             11
sql engineer                             7
ai strategist                            5
data strategy consultant                 4
computer vision specialist               3
Name: count, dtype: int64

In [10]:

# Show only selected columns
preview_df = combined_df[["title", "search_keyword", "normalized_title"]]

# fix text wrapping, and display
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)   # show all rows

# Peek at the first 20 rows
preview_df.head(100)

,title,search_keyword,normalized_title
0,AI Engineer,ai engineer,ai engineer
1,AI Engineer (Contract 1 Year) - EGG Digital,ai engineer,ai engineer
2,AI Engineer (Pathum Thani),ai engineer,ai engineer
3,AI Engineer,ai engineer,ai engineer
4,Responsible AI Engineer,ai engineer,ai engineer
5,Senior AI Engineer (EGG Digital),ai engineer,ai engineer
6,Machine Learning Engineer (Project Algorithmic Trading),ai engineer,machine learning engineer
7,AI Engineer,ai engineer,ai engineer
8,AI Engineer (Innovation Department),ai engineer,ai engineer
9,AI Solutions Analyst,ai engineer,Unclassified


In [11]:
# Show only selected columns
preview_df_unclassified = combined_df.loc[
    combined_df["normalized_title"] == "Unclassified",
    ["title", "search_keyword", "normalized_title"]
].drop_duplicates(subset=["title"])

# fix text wrapping, and display
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)   # show all rows

preview_df_unclassified

preview_df_unclassified.to_csv(
    f"../data/jobstreet_{country}_unclassified.csv", 
    index=False,
    quotechar='"',
    escapechar='\\',
    encoding='utf-8-sig')